In [64]:
import pandas as pd
import nfl_data_py as nfl

In [65]:
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',]

In [66]:
weekly = pd.read_parquet('data/weekly.parquet')
per_game = weekly.groupby(['season','week','recent_team', 'opponent_team'])[stat_cols].sum().reset_index().sort_values(by=['season','week'])
per_game['pass_pct'] = per_game['attempts'] / (per_game['attempts'] + per_game['carries'])

In [75]:
cols2roll = ["completions", "attempts", "carries", "passing_yards", "rushing_yards", "pass_pct", "receiving_yards"]

In [80]:
roll_stats = [x + '_team_roll' for x in cols2roll]
print(roll_stats)
team = per_game.copy()
team[roll_stats] = team.groupby("recent_team")[
    cols2roll
].transform(lambda x: x.rolling(7, min_periods=1).mean()).sort_values(by='completions')
team

['completions_team_roll', 'attempts_team_roll', 'carries_team_roll', 'passing_yards_team_roll', 'rushing_yards_team_roll', 'pass_pct_team_roll', 'receiving_yards_team_roll']


,season,week,recent_team,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,fantasy_points,fantasy_points_ppr,pass_pct,completions_team_roll,attempts_team_roll,carries_team_roll,passing_yards_team_roll,rushing_yards_team_roll,pass_pct_team_roll,receiving_yards_team_roll
0,2010,1,ARI,LA,22,41,297.0,1,0.0,2.0,...,60.779999,82.779999,0.661290,22.000000,41.000000,21.000000,297.000000,112.000000,0.661290,297.000000
1,2010,1,ATL,PIT,27,44,252.0,0,1.0,2.0,...,39.080002,66.080002,0.637681,27.000000,44.000000,25.000000,252.000000,58.000000,0.637681,252.000000
2,2010,1,BAL,NYJ,20,38,248.0,0,1.0,2.0,...,39.619999,59.619999,0.520548,20.000000,38.000000,35.000000,248.000000,49.000000,0.520548,248.000000
3,2010,1,BUF,MIA,18,34,139.0,1,0.0,3.0,...,34.459999,52.459999,0.666667,18.000000,34.000000,17.000000,139.000000,50.000000,0.666667,139.000000
4,2010,1,CAR,NYG,14,35,182.0,1,3.0,4.0,...,36.380001,50.380001,0.593220,14.000000,35.000000,24.000000,182.000000,89.000000,0.593220,182.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,2024,12,SEA,ARI,22,31,254.0,1,1.0,5.0,...,50.060001,72.059998,0.553571,23.714286,35.285714,22.142857,269.714286,79.285714,0.615299,269.714286
8085,2024,12,SF,GB,17,29,199.0,1,1.0,2.0,...,36.259998,53.260002,0.644444,16.571429,25.428571,22.571429,205.428571,114.571429,0.536039,205.428571
8086,2024,12,TB,NYG,24,30,294.0,0,0.0,0.0,...,78.760002,102.760002,0.483871,25.142857,35.000000,27.285714,259.285714,146.428571,0.557602,258.000000
8087,2024,12,TEN,HOU,18,25,278.0,2,1.0,8.0,...,74.119995,92.120003,0.431034,19.428571,31.000000,28.000000,223.428571,122.428571,0.527266,223.428571


In [81]:
opp_stats = [x + '_opp_roll' for x in cols2roll]
opp = per_game.copy()
opp[opp_stats] = opp.groupby("opponent_team")[
    cols2roll
].transform(lambda x: x.rolling(7, min_periods=1).mean()).sort_values(by='completions')
team

,season,week,recent_team,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,fantasy_points,fantasy_points_ppr,pass_pct,completions_team_roll,attempts_team_roll,carries_team_roll,passing_yards_team_roll,rushing_yards_team_roll,pass_pct_team_roll,receiving_yards_team_roll
0,2010,1,ARI,LA,22,41,297.0,1,0.0,2.0,...,60.779999,82.779999,0.661290,22.000000,41.000000,21.000000,297.000000,112.000000,0.661290,297.000000
1,2010,1,ATL,PIT,27,44,252.0,0,1.0,2.0,...,39.080002,66.080002,0.637681,27.000000,44.000000,25.000000,252.000000,58.000000,0.637681,252.000000
2,2010,1,BAL,NYJ,20,38,248.0,0,1.0,2.0,...,39.619999,59.619999,0.520548,20.000000,38.000000,35.000000,248.000000,49.000000,0.520548,248.000000
3,2010,1,BUF,MIA,18,34,139.0,1,0.0,3.0,...,34.459999,52.459999,0.666667,18.000000,34.000000,17.000000,139.000000,50.000000,0.666667,139.000000
4,2010,1,CAR,NYG,14,35,182.0,1,3.0,4.0,...,36.380001,50.380001,0.593220,14.000000,35.000000,24.000000,182.000000,89.000000,0.593220,182.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,2024,12,SEA,ARI,22,31,254.0,1,1.0,5.0,...,50.060001,72.059998,0.553571,23.714286,35.285714,22.142857,269.714286,79.285714,0.615299,269.714286
8085,2024,12,SF,GB,17,29,199.0,1,1.0,2.0,...,36.259998,53.260002,0.644444,16.571429,25.428571,22.571429,205.428571,114.571429,0.536039,205.428571
8086,2024,12,TB,NYG,24,30,294.0,0,0.0,0.0,...,78.760002,102.760002,0.483871,25.142857,35.000000,27.285714,259.285714,146.428571,0.557602,258.000000
8087,2024,12,TEN,HOU,18,25,278.0,2,1.0,8.0,...,74.119995,92.120003,0.431034,19.428571,31.000000,28.000000,223.428571,122.428571,0.527266,223.428571


In [82]:
opp

,season,week,recent_team,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,fantasy_points,fantasy_points_ppr,pass_pct,completions_opp_roll,attempts_opp_roll,carries_opp_roll,passing_yards_opp_roll,rushing_yards_opp_roll,pass_pct_opp_roll,receiving_yards_opp_roll
0,2010,1,ARI,LA,22,41,297.0,1,0.0,2.0,...,60.779999,82.779999,0.661290,22.000000,41.000000,21.000000,297.000000,112.000000,0.661290,297.000000
1,2010,1,ATL,PIT,27,44,252.0,0,1.0,2.0,...,39.080002,66.080002,0.637681,27.000000,44.000000,25.000000,252.000000,58.000000,0.637681,252.000000
2,2010,1,BAL,NYJ,20,38,248.0,0,1.0,2.0,...,39.619999,59.619999,0.520548,20.000000,38.000000,35.000000,248.000000,49.000000,0.520548,248.000000
3,2010,1,BUF,MIA,18,34,139.0,1,0.0,3.0,...,34.459999,52.459999,0.666667,18.000000,34.000000,17.000000,139.000000,50.000000,0.666667,139.000000
4,2010,1,CAR,NYG,14,35,182.0,1,3.0,4.0,...,36.380001,50.380001,0.593220,14.000000,35.000000,24.000000,182.000000,89.000000,0.593220,182.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,2024,12,SEA,ARI,22,31,254.0,1,1.0,5.0,...,50.060001,72.059998,0.553571,23.142857,35.857143,25.142857,243.857143,107.714286,0.591529,243.857143
8085,2024,12,SF,GB,17,29,199.0,1,1.0,2.0,...,36.259998,53.260002,0.644444,20.000000,30.285714,27.000000,206.142857,114.714286,0.528248,206.142857
8086,2024,12,TB,NYG,24,30,294.0,0,0.0,0.0,...,78.760002,102.760002,0.483871,18.714286,26.857143,29.857143,216.142857,164.571429,0.483051,216.142857
8087,2024,12,TEN,HOU,18,25,278.0,2,1.0,8.0,...,74.119995,92.120003,0.431034,20.285714,34.428571,25.142857,246.428571,102.142857,0.572732,246.428571


In [83]:
team[['recent_team','season','week'] + roll_stats].to_csv('data/agg/team_stats.csv')
opp[['opponent_team','season','week'] + opp_stats].to_csv('data/agg/opp_stats.csv')